In [1]:
import pandas as pd
import yfinance as yf
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
from psycopg2.errors import UniqueViolation
from fin_func import *
from fin_func import get_commodities_data 

In [2]:
# Database connection details
database = 'fin_data'
user = 'stevengiallourakis'
password = 'Birdman11!' # this might not be needed depenending on where you are connecting from.
host = 'localhost'
port =  '5433'
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [ ]:
# Details for stock api request

symbol = 'HG=F'
start_date = '2020-01-01'
end_date = '2024-07-04'

In [ ]:
# DB = stocks, table = stocks

stock_data = get_stock_data(symbol, start_date, end_date)

# add data to database - table stocks

data_to_db(stock_data, 'stocks', engine)

print(stock_data)

In [3]:
# Details for commodities api request

symbol = 'HG=F'
start_date = '2020-01-01'
end_date = '2024-07-04'

In [6]:
# DB = stocks, table = commodities
commodities_data = get_commodities_data(symbol, start_date, end_date)

data_to_db(commodities_data, 'commodities', engine)

print(commodities_data)

[*********************100%%**********************]  1 of 1 completed

commodities_data downloaded, High and Low columns removed, daily_change added
           Date    Open   Close  daily_change symbol
0    2020-01-02  2.8165  2.8330        0.0165   HG=F
1    2020-01-03  2.7935  2.7985        0.0050   HG=F
2    2020-01-06  2.7780  2.8005        0.0225   HG=F
3    2020-01-07  2.8010  2.8040        0.0030   HG=F
4    2020-01-08  2.7870  2.8190        0.0320   HG=F
...         ...     ...     ...           ...    ...
1129 2024-06-27  4.3590  4.3285       -0.0305   HG=F
1130 2024-06-28  4.3265  4.3905        0.0640   HG=F
1131 2024-07-01  4.3885  4.4265        0.0380   HG=F
1132 2024-07-02  4.4240  4.4295        0.0055   HG=F
1133 2024-07-03  4.4525  4.5410        0.0885   HG=F

[1134 rows x 5 columns]


In [ ]:
# Bulk Data download

bulk_data = []

In [ ]:
# Bulk Data download

bulk_data = []

# Iterate through list of stocks

for symbol in bulk_data:
        print("Retrieving stock data")
            
        # Retrieve stock data and update dataframe to fit database
        stock_data = get_stock_data(symbol, start_date, end_date)


        # Check if the stock data is successfully downloaded
        if stock_data.empty:
            print(f"Failed to download data for {symbol}")
        else:
            print(f"Stock data downloaded and updated for {symbol}")

    
        # Send data to database
        data_to_db(stock_data, 'stocks', engine)